# RNN 

In [23]:
# import

import matplotlib.pyplot as plt


import numpy as np
np.random.seed(42)

import random
random.seed(42)

import pickle

import tensorflow as tf
tf.random.set_seed(42)

# Désactiver cuDNN non déterministes
tf.config.experimental.enable_op_determinism()

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, mean_absolute_error, root_mean_squared_error
from sklearn.model_selection import KFold 


In [24]:
taille_fenetre_to_run = 20
taille_stride_to_run = 5
# 20w_5s déterminer dans search window size 

In [25]:
data = pickle.load(open(f"Data/donnees_3D_{taille_fenetre_to_run}w_{taille_stride_to_run}s.pkl", "rb"))

In [26]:
data.keys()

dict_keys(['X_np_3D', 'y_np', 'features_names_3D', 'idx_explicabilite', 'X_3D_explicabilite', 'y_explicabilite'])

In [27]:
X_3D = data["X_np_3D"]
y_np = data["y_np"]

In [28]:
X_binary_explicabilite = data["X_3D_explicabilite"]
y_explicabilite = data["y_explicabilite"]

In [29]:
features_3D = data["features_names_3D"]

In [30]:
X_binary_explicabilite.shape

(73, 20, 25)

In [31]:
# Paramètres pour la validation croisée
n_splits = 5
kf = KFold(n_splits=n_splits, random_state=42, shuffle=True)

# Listes pour stocker les scores
mse_scores = []
mae_scores = []
mape_scores = []
rmse_scores = []

# Création d'un modèle RNN
def create_rnn(input_shape):
    model = Sequential([
        #LSTM(64, activation='tanh', input_shape=input_shape, return_sequences=False),
        #Dense(32, activation='relu'),
        SimpleRNN(64, activation='tanh', input_shape=input_shape, return_sequences=False),
        Dense(1)  # Sortie pour régression
    ])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    return model

for fold, (train_index, test_index) in enumerate(kf.split(X_3D)):
    print(f"Fold {fold+1}/{n_splits}")
    X_train, X_test = X_3D[train_index], X_3D[test_index]
    y_train, y_test = y_np[train_index], y_np[test_index]
    
    # Création du modèle
    model = create_rnn(X_train.shape[1:]) 
    
    # Entraînement
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)
    
    # Prédictions
    y_pred = model.predict(X_test) #.flatten()
    
    # Calcul des scores
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    
    mse_scores.append(mse)
    mae_scores.append(mae)
    mape_scores.append(mape)
    rmse_scores.append(rmse)
    
    print(f"MSE: {mse:.2f}")
    print(f"MAE: {mae:.2f}")
    print(f"MAPE: {mape:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print()


Fold 1/5
Epoch 1/10


/Users/zoemarquis/Documents/projet_industrie/projet_batteries/mon_venv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
2025-01-11 22:09:12.998205: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be 

476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 6809.2378 - mae: 81.9074
Epoch 2/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 2764.4863 - mae: 52.0925
Epoch 3/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 987.4781 - mae: 30.8128
Epoch 4/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 279.4230 - mae: 15.8751
Epoch 5/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 69.4495 - mae: 7.7084
Epoch 6/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 28.1573 - mae: 4.6536
Epoch 7/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 23.3579 - mae: 3.8433
Epoch 8/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 23.0489 - mae: 3.6844
Epoch 9/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 23.0352 - mae: 3.6611
Epoch 10/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 23.0349 - mae: 3.6593
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 934us/step


2025-01-11 22:09:20.198054: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}
2025-01-11 22:09:20.198381: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),

MSE: 22.47
MAE: 3.66
MAPE: 0.04
RMSE: 4.74

Fold 2/5
Epoch 1/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 6835.7432 - mae: 82.0817
Epoch 2/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 2800.5127 - mae: 52.4335
Epoch 3/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1007.2668 - mae: 31.1258
Epoch 4/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 287.2932 - mae: 16.1068
Epoch 5/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 71.5476 - mae: 7.8179
Epoch 6/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 28.5331 - mae: 4.7025
Epoch 7/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 23.4622 - mae: 3.8653
Epoch 8/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 23.1376 - mae: 3.6972
Epoch 9/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 23.1249 - mae: 3.6716
Epoch 10/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 23.1251 - mae: 3.6686
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  


...

2025-01-11 22:09:34.279385: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}
2025-01-11 22:09:34.279685: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),

Epoch 1/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 6845.1431 - mae: 82.1344
Epoch 2/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 2794.0374 - mae: 52.3786
Epoch 3/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1003.1039 - mae: 31.0676
Epoch 4/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 284.9758 - mae: 16.0403
Epoch 5/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 70.4829 - mae: 7.7720
Epoch 6/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 28.1508 - mae: 4.6605
Epoch 7/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 23.3559 - mae: 3.8167
Epoch 8/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 23.1082 - mae: 3.6522
Epoch 9/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 23.1088 - mae: 3.6275
Epoch 10/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 23.1099 - mae: 3.6256
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 908us/step
MSE: 24.73
MAE: 3.73
MAPE: 0.04
RMSE: 4.97

Fold 5/5


2025-01-11 22:09:41.148541: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}
2025-01-11 22:09:41.148837: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),

Epoch 1/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 6783.7729 - mae: 81.7352
Epoch 2/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 2718.8096 - mae: 51.6588
Epoch 3/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 962.9227 - mae: 30.4198
Epoch 4/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 269.2981 - mae: 15.5642
Epoch 5/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 66.5406 - mae: 7.5308
Epoch 6/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 27.7490 - mae: 4.6074
Epoch 7/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 23.5506 - mae: 3.8498
Epoch 8/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 23.3522 - mae: 3.7043
Epoch 9/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 23.3541 - mae: 3.6837
Epoch 10/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 23.3546 - mae: 3.6826
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
MSE: 23.26
MAE: 3.68
MAPE: 0.04
RMSE: 4.82



2025-01-11 22:09:48.157928: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}
2025-01-11 22:09:48.158239: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),

In [32]:
# n_splits = 5
# kf = KFold(n_splits=n_splits, random_state=42, shuffle=True) 

# mse_scores = []
# mae_scores = []
# mape_scores = []
# rmse_scores = []

# # déterminer par grid search : 
# # {'bootstrap': True, 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}

# rf = RandomForestRegressor(n_estimators=200, 
#                            min_samples_split=2, 
#                            min_samples_leaf=1, 
#                            max_depth=None, 
#                            bootstrap=True,
#                            random_state=42)

# for fold, (train_index, test_index) in enumerate(kf.split(X_np_binary)):
#     print(f"Running fold {fold+1}/{n_splits}")
#     X_train, X_test = X_np_binary[train_index], X_np_binary[test_index]
#     y_train, y_test = y_np[train_index], y_np[test_index]

#     rf.fit(X_train, y_train)

#     y_pred = rf.predict(X_test)

#     mse = mean_squared_error(y_test, y_pred)
#     mae = mean_absolute_error(y_test, y_pred)
#     mape = mean_absolute_percentage_error(y_test, y_pred)
#     rmse = root_mean_squared_error(y_test, y_pred)

#     mse_scores.append(mse)
#     mae_scores.append(mae)
#     mape_scores.append(mape)
#     rmse_scores.append(rmse)

#     # save the model to disk
#     filename = f'Models/RandomForest_{taille_fenetre_to_run}_fold_{fold}.sav'
#     pickle.dump(rf, open(filename, 'wb'))
    

In [33]:
# min

In [34]:
# print(f"Mean MSE: {np.mean(mse_scores)}")
# # print(f"Mean MAE: {np.mean(mae_scores)}")
# print(f"Mean MAPE: {np.mean(mape_scores)}")
# print(f"Mean RMSE: {np.mean(rmse_scores)}")

In [35]:
# avec reshape : 
# Mean MSE: 0.09505094967631325
# Mean MAE: 0.15491823287857118
# Mean MAPE: 0.0016792438577561838
# Mean RMSE: 0.30705152481328996

In [36]:
# print(f'MSE scores: {mse_scores}')
# print(f'MAE scores: {mae_scores}')
# print(f'MAPE scores: {mape_scores}')
# print(f'RMSE scores: {rmse_scores}')

In [37]:
# # avec reshape 

# MSE scores: [np.float64(0.0861445777390501), np.float64(0.12650759770432213), np.float64(0.07577228909305105), np.float64(0.08608884352657475), np.float64(0.10074144031856817)]
# MAE scores: [np.float64(0.1553254147286415), np.float64(0.15929255096488815), np.float64(0.15014044016561415), np.float64(0.15925454910113407), np.float64(0.15057820943257816)]
# MAPE scores: [np.float64(0.0016787887663007298), np.float64(0.001727112631012994), np.float64(0.0016252426070005458), np.float64(0.0017326526248614877), np.float64(0.0016324226596051616)]
# RMSE scores: [np.float64(0.2935039654571129), np.float64(0.35567906559751605), np.float64(0.2752676680851768), np.float64(0.2934090038266971), np.float64(0.31739792109994697)]

In [38]:
# # get meilleur fold
# best_fold = np.argmin(mse_scores)
# print(f"Best fold: {best_fold}")

# # "Si vous êtes strict sur la validation ou travaillez avec de grandes quantités de données, préférez le meilleur modèle de KFold."

In [39]:
# rf_best = pickle.load(open(f'Models/RandomForest_{taille_fenetre_to_run}_fold_{best_fold}.sav', 'rb'))

In [40]:
# # Récupérer l'importance des caractéristiques
# feature_importances = rf_best.feature_importances_

# # Afficher l'importance des caractéristiques sous forme de graphique
# plt.bar(range(len(feature_importances)), feature_importances)
# plt.xlabel('Caractéristiques')
# plt.ylabel('Importance')
# plt.title('Importance des caractéristiques pour RandomForestRegressor')
# plt.show()


In [41]:
# print(len(features_binary))
# print(X_np_binary.shape)
# print(len(feature_importances))
# # ok tout semble cohérent 

In [42]:
# import shap

# # Créer un explainer SHAP pour RandomForest
# explainer = shap.TreeExplainer(rf_best)

# # # Calculer les valeurs SHAP pour les donnéeschoisies pour l'explicabilité
# shap_values = explainer.shap_values(X_binary_explicabilite)

# # Visualiser les valeurs SHAP pour une prédiction
# shap.initjs()  # Pour activer les visualisations interactives


In [43]:
# # Visualiser les valeurs SHAP avec un graphique résumé
# shap.summary_plot(shap_values, X_binary_explicabilite, feature_names=features_binary, show=False)

# # Sauvegarder la visualisation dans un fichier
# plt.savefig(f"Images/shap_RF_{taille_fenetre_to_run}w_{taille_stride_to_run}s.png")
# plt.show()

In [44]:
# TODO explicabilité : dependance des valeurs ? dependance_plot ?
# (pour les features apparaissant dans le résumé) 

# TODO explicabilité : force_plot 
# 